In [1]:
from app.backend.core.file_processor import FileProcessor
from app.backend.core.analyzer import Analyzer

processor = FileProcessor()
analyzer = await Analyzer.create()


2025-07-08 00:20:59 | app.backend.services.groq | DEBUG | Retrieving Groq API key from Azure Key Vault...
2025-07-08 00:21:02 | app.backend.services.groq | DEBUG | Groq API key retrieved.


In [2]:
resume = await processor.extract_from_file("data/ExampleResume1.pdf")
response = await analyzer.analyze_candidate(resume)

2025-07-08 00:21:05 | app.backend.core.file_processor | INFO | Extracting text from local file: data/ExampleResume1.pdf
2025-07-08 00:21:05 | app.backend.core.file_processor | INFO | Using .PDF parser for file: data/ExampleResume1.pdf
2025-07-08 00:21:05 | app.backend.core.file_processor | INFO | Extracting text from 'data/ExampleResume1.pdf' using local PDF parser (pypdf)
2025-07-08 00:21:05 | app.backend.services.groq | INFO | Running structured query with function calling using ainvoke...
2025-07-08 00:21:08 | app.backend.services.groq | DEBUG | Beautified JSON args:
{
    "candidate_name": "Hedy Lamarr",
    "certificate": [],
    "comment": "Hedy Lamarr has a strong background in computer science and statistics, with experience in data engineering and software development. She has worked with various technologies and has a proven track record of delivering results in fast-paced environments.",
    "education": [{
        "field": "Computer Science, Statistics",
        "institutio

In [1]:
from app.backend.services.factory import AIServiceFactory

service = await AIServiceFactory.create_service()
response = await service.query("What is the area of the UK?")
print(response)

2025-07-07 00:38:58 | app.backend.services.groq | DEBUG | Retrieving Groq API key from Azure Key Vault...
2025-07-07 00:39:01 | app.backend.services.groq | DEBUG | Groq API key retrieved.


Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x00000255A5E7B1C0>


2025-07-07 00:39:01 | app.backend.services.groq | INFO | Calling Groq model 'llama3-70b-8192' using ainvoke...
The total area of the United Kingdom (UK) is approximately 243,610 square kilometers (93,625 square miles). This includes:

* England: 130,279 km² (50,261 sq mi)
* Scotland: 78,772 km² (30,414 sq mi)
* Wales: 20,779 km² (8,023 sq mi)
* Northern Ireland: 14,160 km² (5,470 sq mi)

Note: These figures are approximate and may vary slightly depending on the source.


In [13]:
from datetime import date

prompt = f"""
You are an expert HR recruiter and resume screener. Your task is to extract relevant information from a resume and organize it into structured categories using the format below.

---

**Categories**:

1. **Synopsis** - A personal summary, description, or “about me” section.  
2. **Experience** - Includes jobs/internships under `<job>` and standalone projects under `<project>`.  
3. **Education** - Each degree appears under a `<degreeEntry>` with optional GPA and graduation year.  
4. **Skills** - Divided into technical and non-technical.

---

**Important Instructions**:
- If no synopsis is present, insert the word `Unknown` in the `<synopsis>` tag.
- Do **not** invent or infer missing information. Only extract what is explicitly present.
- Use the exact XML structure and tag names provided.
- All durations must use time units (e.g., `"3 months"`, `"2 years"`).
- Only include UTF-8 characters. Strip bullets, markdown, or decorative formatting.
    - If only a single month is present in the date range, the duration is `"1 month"`
    - For reference, `"Present"` in a job duration refers to {date.today()}

---

**Format your output as follows**:

```xml
<resume>
    <synopsis>
        INSERT SYNOPSIS INFORMATION HERE
    </synopsis>
    <experience>
        <job>
            <company>
                INSERT COMPANY NAME
            </company>
            <duration>
                INSERT JOB DURATION
            </duration>
            <description>
                INSERT JOB DESCRIPTION
            </description>
        </job>
        <!-- Repeat <job> as needed -->

        <project>
            <name>
                INSERT PROJECT NAME
            </name>
            <duration>
                INSERT PROJECT DURATION (if available)
            </duration>
            <description>
                INSERT PROJECT DESCRIPTION
            </description>
        </project>
        <!-- Repeat <project> as needed -->
    </experience>
    <education>
        <degreeEntry>
            <university>
                INSERT UNIVERSITY NAME
            </university>
            <degree>
                <level>
                    INSERT DEGREE LEVEL (e.g., Doctorate, Masters, Baccalaureate)
                </level>
                <name>
                    INSERT DEGREE NAME (e.g., Computer Science)
                </name>
            </degree>
            <graduationYear>
                INSERT GRADUATION YEAR (if available)
            </graduationYear>
            <gpa>
                INSERT GPA (if available)
            </gpa>
        </degreeEntry>
        <!-- Repeat <degreeEntry> as needed -->
    </education>
    <skills>
        <technical>
            <software>
                LIST SOFTWARE SKILLS
            </software>
            <languages>
                LIST PROGRAMMING LANGUAGES
            </languages>
            <frameworks>
                LIST FRAMEWORKS
            </frameworks>
            <tools>
                LIST TECHNICAL TOOLS
            </tools>
        </technical>
        <nontechnical>
            LIST NON-TECHNICAL SKILLS
        </nontechnical>
    </skills>
</resume>

---

You will now be given a resume in plain text. Extract and organize the information as described above.
**Resume**:  
{resume}
"""